In [44]:
import pandas as pd
movie = pd.read_csv("../db/movie_list.csv", index_col=0)
movie

,영화ID,영화명,개봉일,국적,장르,감독,배우
0,0,범죄도시2,2022-05-18,한국,"범죄,액션",이상용,"마동석,손석구,최귀화,박지환,허동원,하준,정재광,남문철,박지영,이주원,음문석,김찬형..."
1,1,아바타: 물의 길,2022-12-14,미국,"액션,어드벤처,SF,스릴러",제임스 카메론,"조 샐다나,샘 워싱턴,시고니 위버,스티븐 랭,케이트 윈슬렛"
2,2,탑건: 매버릭,2022-06-22,미국,액션,조셉 코신스키,"톰 크루즈,마일즈 텔러,제니퍼 코넬리,존 햄,에드 해리스,글렌 포웰,제이 엘리스,그..."
3,3,스파이더맨: 노 웨이 홈,2021-12-15,미국,"액션,어드벤처,SF",존 왓츠,"톰 홀랜드,젠데이아 콜먼,베네딕트 컴버배치,제이콥 배덜런,존 파브로,마리사 토메이"
4,4,한산: 용의 출현,2022-07-27,한국,"액션,드라마",김한민,"박해일,이지영,이정태,변요한,안성기,손현주,김성규,김성균,김향기,옥택연,공명,박지환..."
...,...,...,...,...,...,...,...
15839,15839,허수아비들의 땅,2009-04-30,한국,드라마,노경태,"김선영,박원주,김화정,김두홍,신예슬,황상연,박현진,서민정,최경락,최광수,윤심경,임독..."
15840,15840,혹성탈출: 진화의 시작,2011-08-17,미국,"액션,SF,드라마",루퍼트 와이어트,"브라이언 콕스,제임스 프랭코,프리다 핀토"
15841,15841,환상의 빛,2016-07-07,일본,드라마,고레에다 히로카즈,"마키코 에수미,나이토 타카시,아사노 타다노부"
15842,15842,후세: 말하지 못한 내 사랑,2013-03-28,일본,"애니메이션,드라마",미야지 마사유키,"코토부키 미나코,미야노 마모루,코니시 카츠유키"


In [45]:
movie_list = movie['영화명']
movie_list

0                  범죄도시2
1              아바타: 물의 길
2                탑건: 매버릭
3          스파이더맨: 노 웨이 홈
4              한산: 용의 출현
              ...       
15839           허수아비들의 땅
15840       혹성탈출: 진화의 시작
15841              환상의 빛
15842    후세: 말하지 못한 내 사랑
15843     흐르는 강물을 어찌 막으랴
Name: 영화명, Length: 15844, dtype: object

In [34]:
import requests
from bs4 import BeautifulSoup
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver import ActionChains
from tqdm import tqdm
from urllib.parse import quote

In [35]:
movie = '범죄도시2'

In [36]:
url = "https://www.naver.com/"
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Edge('edgedriver', options=options)

driver.get(url)
time.sleep(2)

C:\Users\gram15\AppData\Local\Temp\ipykernel_12108\1222300948.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('edgedriver', options=options)


In [37]:
driver.find_element(By.CSS_SELECTOR, '#query').send_keys([f'영화 {movie} 평점', Keys.ENTER])
time.sleep(1)

In [75]:
# 리뷰/평점 리스트에서 관람객으로 설정되어있을 경우 전체 리뷰로 변경
soup = BeautifulSoup(driver.page_source, 'html.parser')

if  soup.select_one('._sorting_select_actual._trigger > a > span.menu._trigger_text').get_text() == '관람객':
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual._trigger').click()
    time.sleep(0.2)
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual > div > div > div > div > ul > li:nth-child(1) > a').click()
else:
    pass

if  soup.select_one('._sorting_select_order > a > span.menu._trigger_text').get_text() == '최신순':
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_order._trigger').click()
    time.sleep(0.2)
    driver.find_element(By.CSS_SELECTOR, '._sorting_select_order > div > div > div > div > ul > li:nth-child(1) > a').click()
else:
    pass

# 스포일러 허용 버튼
driver.find_element(By.CSS_SELECTOR, '._spoiler_switch > button.btn_area_auto').click()

In [39]:
scroll = driver.find_element(By.CSS_SELECTOR, '._cm_content_area_rating > div > div:nth-child(2) > div.lego_review_list._scroller')

while True :
	# 스크롤 전의 마지막 리뷰 내용
	prev_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
	# 첫번째로 스크롤 내리기
	driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", scroll)
	# 시간대기				
	time.sleep(0.5)
	# 스크롤 후 마지막 리뷰 내용
	soup = BeautifulSoup(driver.page_source, 'html.parser')
	current_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
	time.sleep(0.5)
	# 스크롤 전과 후의 마지막 리뷰가 같으면 종료
	if current_review == prev_review:
		False
		break

In [55]:
# 리뷰만 추출
soup = BeautifulSoup(driver.page_source, 'html.parser')
review_list = soup.select("div.lego_review_list._scroller > ul > li > div.area_review_content > div > span.desc._text")
print(len(review_list))

90


In [70]:
review_list[0:5]

[<span class="desc _text">이 영화의 속편은 100% 성공이다. 원래 한국영화들 속편은 잘 안되고 속편 나오면 망하는 케이스던데 이번 범죄도시2는 속편 성공함 오랜만에 극장에서 다같이 웃으면서 봄 장이수 개웃기고 ㅋㅋㅋ 이번에도 명장면 + 명대사 많이 나온다 이 영화 꼭 봐라 </span>,
 <span class="desc _text">시리즈로 계속 나왔으면 좋겠다. 마동석한테 최적화된 작품이다. </span>,
 <span class="desc _text">니가 강해상이냐? 아뇨. 구씬데요? </span>,
 <span class="desc _text">전편을 보고 가야 장이수 얼굴만 봐도 웃음이 나오는 이유를 알 수 있습니다:) </span>,
 <span class="desc _text">와…. 손석구 연기 진짜 미쳣다…오바하는 범죄자연기가 아니라 ㄹㅇ 범죄자같음 </span>]

In [68]:
for org in review_list:
    review_before = str(org)
    fwd = review_before.split('">')[1]
    bck = fwd.split(' </')[0]
    print(bck)

이 영화의 속편은 100% 성공이다. 원래 한국영화들 속편은 잘 안되고 속편 나오면 망하는 케이스던데 이번 범죄도시2는 속편 성공함 오랜만에 극장에서 다같이 웃으면서 봄 장이수 개웃기고 ㅋㅋㅋ 이번에도 명장면 + 명대사 많이 나온다 이 영화 꼭 봐라
시리즈로 계속 나왔으면 좋겠다. 마동석한테 최적화된 작품이다.
니가 강해상이냐? 아뇨. 구씬데요?
전편을 보고 가야 장이수 얼굴만 봐도 웃음이 나오는 이유를 알 수 있습니다:)
와…. 손석구 연기 진짜 미쳣다…오바하는 범죄자연기가 아니라 ㄹㅇ 범죄자같음
액션과 유머 적절한 5:5였다. 꼭 사운드 좋은 곳에서 보길
장첸따위는 잊어라 구씨가 온다!!
잼써잼써ㅠ 마블리 펀치 사운드 장난 아니니까 꼭 극장에서 보세요ㅋㅋ
통쾌함을 마블에서 못보고 여기서 보네요. 스토리도 깔끔해서 몰입 하나도 안끊깁니다웃긴 요소가 기대벽이 있었음에도 꽤 있어요 개인적으로 조금 더 줘패고 싶었네요
이건 크게 잘될만하다.
진짜 쩐다. 특히 손석구 분노 연기 부들부들 떠는 게 디지고. 오랜만에 한국 영화 제대로 봤다. 천만 거뜬할 듯
1편 윤계상 악역 하는거 보고 역대급 이라 생각했는데 손석구가 그걸 넘어선 느낌;;ㄷㄷ
니 내 누군지 아늬? 나 하얼빈에 장첸이야
1편이 꽤나 수작이라 내심 걱정스러운 마음으로 봤는데 너무 재미져요 2!.3편도 기대!
솔직하게 장첸보다 강해상이 더 강해보인다
아.. 범죄자가 그렇게 섹시하면 어떡해요. 반해버렸네
누가 5야 ? 저도 순간 고민했습니다
와 한국영화의 속편 중에 처음으로 실망 안했다..
구씨가 산포시로 도망친 이유
주인공이 절대 걱정이 안되는 영화.
 1보다 재밌는게 말이되나여??진심 ㅋㅋㅋㅋ명장면 많고 웃긴장면도 많곸ㅋㅋㅋ감탄해씀 누구의 머릿속에서 나온아이디어인짘ㅋㅋ특히 에스컬레이터 씬ㅋㅋㅋㅋ그냥 보세여!!밑에 누가 킬링타임용이라고 적었던데 장난하나요..긴장감에 웃긴장면에 지루할 틈이 없습니다 ..마블리 너무...</span>
설마 재밌을까 했는데,개인적으로 2가 더 재밌었어요.
손석구.... 연기 너무

In [43]:
# 평점 요소
rating = int(soup.select_one('li:nth-child(3) > div.area_title_box > div > div.area_text_box').get_text().split(')')[-1])
rating

8

In [107]:
url = "https://www.naver.com/"
options = webdriver.EdgeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")
driver = webdriver.Edge('edgedriver', options=options)


driver.get(url)
time.sleep(2)

rating_before = [] # 평점 임시 리스트
review_before = [] # 리뷰 임시 리스트
rating_list = []
review_list = []
movie_title = []

for movie in movie_list:
    try:
        driver.find_element(By.CSS_SELECTOR, '#query').send_keys([f'영화 {movie} 평점', Keys.ENTER])
        time.sleep(1)
    except:
        driver.find_element(By.CSS_SELECTOR, '#nx_query').send_keys([f'영화 {movie} 평점', Keys.ENTER])
        time.sleep(1)

    # 리뷰/평점 리스트에서 관람객으로 설정되어있을 경우 전체 리뷰로 변경
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    time.sleep(1)
    if  soup.select_one('._sorting_select_actual._trigger > a > span.menu._trigger_text').get_text() == '관람객':
        driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual._trigger').click()
        time.sleep(0.5)
        driver.find_element(By.CSS_SELECTOR, '._sorting_select_actual > div > div > div > div > ul > li:nth-child(1) > a').click()
    else:
        pass

    if  soup.select_one('._sorting_select_order > a > span.menu._trigger_text').get_text() == '최신순':
        driver.find_element(By.CSS_SELECTOR, '._sorting_select_order._trigger').click()
        time.sleep(0.5)
        driver.find_element(By.CSS_SELECTOR, '._sorting_select_order > div > div > div > div > ul > li:nth-child(1) > a').click()
    else:
        pass

    # 스포일러 허용 버튼
    driver.find_element(By.CSS_SELECTOR, '._spoiler_switch > button.btn_area_auto').click()

    # 셀레니움 스크롤 끝까지 내려도 계속 내리는 페이지라면
    scroll = driver.find_element(By.CSS_SELECTOR, '._cm_content_area_rating > div > div:nth-child(2) > div.lego_review_list._scroller')

    while True :
        # 스크롤 전의 마지막 리뷰 내용
        prev_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
        # 첫번째로 스크롤 내리기
        driver.execute_script("arguments[0].scrollBy(0, document.body.scrollHeight)", scroll)
        # 시간대기				
        time.sleep(1)
        # 스크롤 후 마지막 리뷰 내용
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        current_review = soup.select("div.lego_review_list._scroller > ul > li")[-1:]
        time.sleep(1)
        # 스크롤 전과 후의 마지막 리뷰가 같으면 종료
        if current_review == prev_review:
            False
            break
    time.sleep(1)
    print("=" * 50)
    print("영화제목 : ", movie)
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 평점 요소
    rating_before = soup.select('div.area_title_box > div > div.area_text_box')
    time.sleep(0.5)
    for org_rv in rating_before:
        rating_org = str(org_rv)
        fwd_del_rv = rating_org.split('중)</span>')[1]
        bck_del_rv = fwd_del_rv.split('</div')[0]
        rating_list.append(int(bck_del_rv))
    print("평점개수 : ", len(rating_before))

    # 리뷰만 추출
    review_before = soup.select("div.lego_review_list._scroller > ul > li > div.area_review_content > div > span.desc._text")
    time.sleep(0.5)
    for org in review_before:
        review_org = str(org)
        fwd_del = review_org.split('">')[1]
        bck_del = fwd_del.split(' </')[0]
        review_list.append(bck_del)
        movie_title.append(movie) # 리뷰 구분을 위한 영화 타이틀 추가
    print("리뷰개수 : ", len(review_before))
    print("=" * 50)
    review_before = [] # 리뷰 임시 리스트 초기화
    url = driver.current_url
    driver.find_element(By.CSS_SELECTOR, '#nx_query').clear()


C:\Users\gram15\AppData\Local\Temp\ipykernel_12108\4257098095.py:6: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Edge('edgedriver', options=options)


영화제목 :  범죄도시2
평점개수 :  20
리뷰개수 :  20
영화제목 :  아바타: 물의 길
평점개수 :  10
리뷰개수 :  10
영화제목 :  탑건: 매버릭
평점개수 :  30
리뷰개수 :  30
영화제목 :  스파이더맨: 노 웨이 홈
평점개수 :  50
리뷰개수 :  50
영화제목 :  한산: 용의 출현
평점개수 :  30
리뷰개수 :  30
영화제목 :  공조2: 인터내셔날
평점개수 :  10
리뷰개수 :  10
영화제목 :  닥터 스트레인지: 대혼돈의 멀티버스
평점개수 :  20
리뷰개수 :  20
영화제목 :  헌트
평점개수 :  210
리뷰개수 :  210
영화제목 :  모가디슈
평점개수 :  20
리뷰개수 :  20
영화제목 :  올빼미
평점개수 :  20
리뷰개수 :  20
영화제목 :  영웅
평점개수 :  50
리뷰개수 :  50
영화제목 :  이터널스
평점개수 :  20
리뷰개수 :  20
영화제목 :  블랙 위도우
평점개수 :  40
리뷰개수 :  40
영화제목 :  가디언즈 오브 갤럭시: Volume 3
평점개수 :  20
리뷰개수 :  20
영화제목 :  쥬라기 월드: 도미니언
평점개수 :  30
리뷰개수 :  30
영화제목 :  마녀(魔女) Part2. The Other One
평점개수 :  40
리뷰개수 :  40
영화제목 :  토르: 러브 앤 썬더
평점개수 :  40
리뷰개수 :  40
영화제목 :  분노의 질주: 더 얼티메이트
평점개수 :  20
리뷰개수 :  20
영화제목 :  미니언즈2
평점개수 :  20
리뷰개수 :  20
영화제목 :  싱크홀
평점개수 :  70
리뷰개수 :  70
영화제목 :  베놈 2: 렛 데어 비 카니지
평점개수 :  30
리뷰개수 :  30
영화제목 :  블랙 팬서: 와칸다 포에버
평점개수 :  40
리뷰개수 :  40
영화제목 :  비상선언
평점개수 :  20
리뷰개수 :  20
영화제목 :  크루엘라
평점개수 :  40
리뷰개수 :  40
영화제목 :  육사오(6/45)
평점개수 :  30


KeyboardInterrupt: 

In [ ]:
result = pd.DataFrame([rating_list, review_list, movie_title], columns=['rating', 'review', 'title'], index=False)